# Trainer
一旦将模型定义好，就可以使用Trainer类来训练模型。Trainer类是PyTorch Lightning的核心类，它负责训练、验证、测试、预测、调整学习率、保存检查点、记录日志等等。Trainer类的构造函数有很多参数，但是大多数参数都有默认值，所以我们可以只设置一些必要的参数。

In [ ]:
from pytorch_lightning import Trainer

model = MyLightningModule()

trainer = Trainer()
trainer.fit(model, train_dataloader, val_dataloader)

## 使用Python脚本
在Python脚本中，建议你使用`main`函数调用`Trainer`。

from argparse import ArgumentParser


def main(hparams):
    model = LightningModule()
    trainer = Trainer(accelerator=hparams.accelerator, devices=hparams.devices)
    trainer.fit(model)


if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument("--accelerator", default=None)
    parser.add_argument("--devices", default=None)
    args = parser.parse_args()

    main(args)

Trainer的`add_argparse_args()`方法已经定义好了其所需要的参数，可以直接调用。

In [ ]:
from argparse import ArgumentParser


def main(args):
    model = LightningModule()
    trainer = Trainer.from_argparse_args(args)
    trainer.fit(model)


if __name__ == "__main__":
    parser = ArgumentParser()
    parser = Trainer.add_argparse_args(parser)
    args = parser.parse_args()

    main(args)

可以通过这种方式运行
python main.py --accelerator 'gpu' --devices 2 --max_steps 10 --limit_train_batches 10 --any_trainer_arg x

## Validation
可以调用`validate()`方法，在训练循环之外对验证集进行评估。如果要在模型初始化或训练后从模型收集新指标，这可能很有用。

In [ ]:
trainer.validate(model=model, dataloaders=val_dataloaders)

## Testing
一旦你完成了训练，随时可以运行测试集！（仅在发表论文或推出产品之前）

In [ ]:
trainer.test(dataloaders=test_dataloaders)

## 重复性
为了在不同的运行中确保重复性，需要设置随机种子，并在`Trainer`中设置`deterministic`参数。

In [ ]:
from pytorch_lightning import Trainer, seed_everything

seed_everything(42, workers=True)
# sets seeds for numpy, torch and python.random.
model = Model()
trainer = Trainer(deterministic=True)

## Trainer flags
auto_scale_batch_size: 自动尝试找到适合显存的最大的batch size。

In [ ]:
# default used by the Trainer (no scaling of batch size)
trainer = Trainer(auto_scale_batch_size=None)

# run batch size scaling, result overrides hparams.batch_size
trainer = Trainer(auto_scale_batch_size="binsearch")

# call tune to find the batch size
trainer.tune(model)

auto_lr_find: 使用`trainer.tune()`的时候运行学习率寻找算法，用来找到最优的初始化学习率。

In [ ]:
# run learning rate finder, results override hparams.learning_rate
trainer = Trainer(auto_lr_find=True)

# call tune to find the lr
trainer.tune(model)

callbacks: 添加一个`Callback`对象列表，在合适的时候会调用它。如`ModelCheckpoint`。
devices: 指定用来训练的设备或其个数或者`auto`。

In [ ]:
# If your machine has GPUs, it will use all the available GPUs for training
trainer = Trainer(devices="auto", accelerator="auto")

# Training with CPU Accelerator using 1 process
trainer = Trainer(devices="auto", accelerator="cpu")

# Training with TPU Accelerator using 8 tpu cores
trainer = Trainer(devices="auto", accelerator="tpu")

# Training with IPU Accelerator using 4 ipus
trainer = Trainer(devices="auto", accelerator="ipu")

enable_checkpointing:默认会在当前工作文件夹保存断点，可以通过设为`False`关闭自动保存。

In [ ]:
# default used by Trainer, saves the most recent model to a single checkpoint after each epoch
trainer = Trainer(enable_checkpointing=True)

# turn off automatic checkpointing
trainer = Trainer(enable_checkpointing=False)

logger: 记录器
max_epochs:
max_steps:
max_time: 
enable_progress_bar: 进度条
resume_from_checkpoint: 从断点恢复模型